In [ ]:
# import Pkg
# Pkg.gc()
# run(`$(Mycelia.MAMBA) clean --all -y`)

# conda   # Get a list of your environments
# conda env remove --name <env1> <env2> ... 

# run(`$(Mycelia.MAMBA) env list`)
# run(`$(Mycelia.MAMBA) env remove --name deepvariant`)
# run(`$(Mycelia.MAMBA) env remove --name gatk4`)
# run(`$(Mycelia.MAMBA) env remove --name medaka`)
# run(`$(Mycelia.MAMBA) env remove --name nanocaller`)
# run(`$(Mycelia.MAMBA) env remove --name polypolish`)
# run(`$(Mycelia.MAMBA) env remove --name picard`)
# run(`$(Mycelia.MAMBA) env remove --name nanovar`)
# run(`$(Mycelia.MAMBA) env remove --name nanoq`)

In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
# @assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
    "FASTX",
    "ProgressMeter",
    # "StatsBase",
    # "Distributions",
    # "StatsPlots",
    # "Random",
    # "Dates",
    "DataFrames",
    "uCSV"
    # "BioSequences",
    # "Conda",
    # "Downloads"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
import Mycelia

In [ ]:
# Pkg.build("Mycelia")

In [ ]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

In [ ]:
# run(`$(Mycelia.MAMBA) env remove --name cactus`)
# Mycelia.add_bioconda_env("cactus")

In [ ]:
reference_assemblies = filter(x -> occursin(r"\.fna$", x) && !occursin("normalized", x), readdir(genome_dir))

In [ ]:
reference_variant_assemblies = filter(x -> occursin(r"\.fna$", x) && occursin("normalized", x) && !occursin("joint", x), readdir(genome_dir))

In [ ]:
reference_assemblies_by_id = Dict(first(split(x, '.')) => x for x in reference_assemblies)

In [ ]:
reference_variant_assemblies_by_id = Dict(first(split(x, '.')) => x for x in reference_variant_assemblies)

In [ ]:
identifiers = intersect(keys(reference_assemblies_by_id), keys(reference_variant_assemblies_by_id))

In [ ]:
identifiers = sort(collect(identifiers), by=x->length(x))

In [ ]:
# identifier = identifiers[1]
# identifier = identifiers[2]
# identifier = identifiers[3]
# identifier = identifiers[4]
# identifier = identifiers[5]
# todo
identifier = identifiers[6]

table = DataFrames.DataFrame(
    samples = ["REFERENCE", "ALTERNATE"],
    file_paths = [reference_assemblies_by_id[identifier],reference_variant_assemblies_by_id[identifier]]
)
cactus_config_file = "$(genome_dir)/$(identifier)-cactus-config.txt"

uCSV.write(data=collect(DataFrames.eachcol(table)), cactus_config_file, header=missing, delim='\t')

CORES = 8
MEMORY = 64

# cmd = 
# `$(Mycelia.MAMBA) run --live-stream -n cactus cactus-pangenome 
# --mgCores $CORES
# --mapCores $CORES
# --consCores $CORES
# --indexCores $CORES
# --mgMemory $MEMORY
# --consMemory $MEMORY
# --indexMemory $MEMORY
# --maxMemory $MEMORY
# --gbz
# --gfa
# --vcf
# --odgi
# --reference REFERENCE
# --outName cactus-test-prefix
# --outDir ./cactus-test-out
# ./cactus-job-store cactus-test.txt`

# rm(joinpath(genome_dir, "cactus-job-store"), recursive=true)
cmd = `podman-hpc run -it -v $(genome_dir):/app -w /app quay.io/comparative-genomics-toolkit/cactus:v2.7.2 cactus-pangenome
--mgCores $CORES
--mapCores $CORES
--consCores $CORES
--indexCores $CORES
--mgMemory $(MEMORY)Gb
--consMemory $(MEMORY)Gb
--indexMemory $(MEMORY)Gb
--maxMemory $(MEMORY)Gb
--gbz
--gfa
--vcf
--odgi
--reference REFERENCE
--outName $(identifier)-cactus
--outDir $(identifier)-cactus
./cactus-job-store $(basename(cactus_config_file))`
run(cmd)
isdir(joinpath(genome_dir, "cactus-job-store")) && rm(joinpath(genome_dir, "cactus-job-store"), recursive=true)